**1.矩阵乘法的例子（尚未理解）**可能我对shared memory有误解

In [3]:
#一个低效率的矩阵乘法的CUDA kernel，因为相同的矩阵元素会被device内存中被反复load很多次
from numba import cuda

@cuda.jit
def matmul(A,B,C): #A,B,C为C=A*B
    i,j=cuda.grid(2) #返回线程的在x和y方向的位置
    if i<C.shape[0] and j<C.shape[1]: #申请的多余的线程舍弃掉不用
        tmp=0
        for k in range(A.shape[1]): #在A的列与B的行上循环
            tmp+=A[i,k]*B[k,j] #矩阵乘法的算法
        C[i,j]=tmp #思想：只需从矩阵的单一元素的视角出发
#总是从矩阵单一元素的视角出发是正确的理解方式

In [1]:
#基于blocked的高效率的矩阵乘法的CUDA kernel，
#blocked算法是利用block内的shared memory来减少对device memory的访问，从而加速。
from numba import cuda, float32
#block内线程为16*16
TPB = 16
@cuda.jit
def fast_matmul(A,B,C):
    SA=cuda.shared.array(shape=(TPB,TPB),dtype=float32) #定义block内shared memory的array，array的size
    SB=cuda.shared.array(shape=(TPB,TPB),dtype=float32) #和type必须设定，注意关键字shape为元组
    x,y=cuda.grid(2)
    tx=cuda.threadIdx.x #二维block上
    ty=cuda.threadIdx.y #线程的索引
    bpg=cuda.gridDim.x #每个grid内x方向上block的数量，为何只有x方向上的block数量？可能我还是不理解shared memory吧
    if x>=C.shape[0] and y>=C.shape[1]:
        return #当线程索引超过矩阵的最大尺寸时，结束运算，即申请的多余的线程舍弃掉不用
    tmp=0
    for i in range(bpg): #对x方向上的每个block循环，为何只有x方向上的block数量？可能我还是不理解shared memory吧
        SA[tx,ty]=A[x,ty+i*TPB] #预加载数据至shared memory。需要画图再理解一下这个地址的对应关系？
        SB[tx,ty]=B[tx+i*TPB,y]
        cuda.syncthreads() #等待所有线程完成预载入，强制同步
        for j in range(TPB): #对block内的线程进行计算，这步可以利用shared memory加速
            tmp+=SA[tx,j]*SB[j,ty]
        cuda.syncthreads() #在进入下一个block的计算之前等待当前block内所有线程完成计算，强制同步
    C[x,y]=tmp